In [1]:
import os
from openai import OpenAI
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)
import os
from langchain_core.messages import SystemMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

In [7]:
prompt = ChatPromptTemplate.from_messages([
            ("system", """Your name is  Assistant. You are a personal assistant .You help users to learn  fundamentals and techniques with detailed answers. 
              helps people to practice  with the help of a projector and a indoor green platform so you might be asked to project a  practice on green.
             if you do not know the answer, do not make up an answer, say that you do not know the answer.
             Answer within 3 sentences to the user's question based on the context: {context}"""),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human","{input}")
            ])
llm = ChatOpenAI(model_name=finetuned_model, temperature=0)
def normal_llm(user_input):
    
    llm_chain = LLMChain(prompt=user_input, llm=llm)
    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

    
    chain = prompt | llm_chain

    return chain

normal_llm("where is japan")

ValidationError: 1 validation error for LLMChain
prompt
  value is not a valid dict (type=type_error.dict)

In [3]:
os.environ["OPENAI_API_KEY"] = ""
job_id = 'ftjob-hjLV1XIM6ccPjzx6zTpAYw5z'

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
finetuned_job = client.fine_tuning.jobs.retrieve(job_id)
finetuned_model = finetuned_job.fine_tuned_model

loader = PyPDFLoader("../_RAG_3.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=80)
docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [4]:
llm = ChatOpenAI(model_name=finetuned_model, temperature=0)
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferWindowMemory(
    llm=llm,
    output_key='answer',
    memory_key='chat_history',
    return_messages=True)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    memory=memory,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    get_chat_history=lambda h : h,
    verbose=False)

In [54]:
template = """
    Your name is  Assistant. You are a personal assistant . Recommend ideal  exercises to help users to improve their  skills.
    You help users to learn  fundamentals and techniques with detailed answers. You also enlight users about  products. Give answers with 2 or 3 sentences, not longer.
    """
prompt = PromptTemplate.from_template(template)
from langchain.prompts.chat import SystemMessagePromptTemplate
chain.combine_docs_chain.llm_chain.prompt.messages[0] = SystemMessagePromptTemplate(prompt=prompt)

In [57]:
response = chain({"question": "what are the  products"})
response

{'question': 'what are the  products',
 'chat_history': [HumanMessage(content='what is  X?'),
  AIMessage(content=' X is a professional  and training system. It is a great choice . It offers a wide range of features to improve your  skills. '),
  HumanMessage(content='what is  X?'),
  AIMessage(content=' X is a high-end  solution for indoor and outdoor practice. It offers real-time green visualization, allowing you to see the ideal putt and your actual putt simultaneously. With its advanced features,  X is a great choice for serious golfers looking to improve their  performance.')],
 'answer': ' offers a range of products for different needs. The home series is perfect for home use, while the professional series is great for coaches and pros. The custom series allows for personalized solutions.',
 'source_documents': [Document(page_content='correctly\nwith\nyour\nputter\nsquare\nto\nyour\ntarget,\nyou\nare\nlikely\nto\nhit\nyour\nputt\ndown\nyour\ntarget\nline.\nThis\nis\nwhy\nit\nis\n